In [58]:
import cv2
import numpy as np
import os

In [59]:
sequence_folder = 'sequence/'
frame_files = sorted([f for f in os.listdir(sequence_folder) if f.endswith('.jpg') or f.endswith('.png')])
initial_bbox = (246, 208, 23, 23) 

initial_hypothesis = np.array([initial_bbox[1], initial_bbox[0], 
                                   initial_bbox[1] + initial_bbox[3], 
                                   initial_bbox[0] + initial_bbox[2]])
frames = (cv2.imread(os.path.join(sequence_folder, f)) for f in frame_files)
frame = list(frames)[0]

In [60]:
top, left, bottom, right = initial_hypothesis

In [61]:
object_mask = np.zeros(frame.shape[:2], dtype=bool)
object_mask[top:bottom, left:right] = True


In [62]:
padding = 20
surrounding_top = max(0, top - padding)
surrounding_left = max(0, left - padding)
surrounding_bottom = min(frame.shape[0], bottom + padding)
surrounding_right = min(frame.shape[1], right + padding)
surrounding_mask = np.zeros(frame.shape[:2], dtype=bool)
surrounding_mask[surrounding_top:surrounding_bottom, surrounding_left:surrounding_right] = True
surrounding_mask[top:bottom, left:right] = False  # exclude the object region

In [63]:
def compute_histogram(image, region_mask, bins):
    histogram = np.zeros(bins)
    # histogram = cv2.calcHist([image], [0,1,2], region_mask.astype(np.uint8), [bins,bins,bins], [0, 256,0, 256,0, 256,0, 256]).flatten()
    print(histogram.shape)

    for c in range(image.shape[-1]):  
        hist =cv2.calcHist([image], [c], region_mask.astype(np.uint8), [bins], [0, 256]).flatten()
        histogram += hist
    return histogram
    # histogram = cv2.calcHist([image], [0], region_mask.astype(np.uint8), [bins], [0, 256]).flatten()

In [64]:
bins = 10

In [65]:
H_O = compute_histogram(frame, object_mask, bins) 
H_S = compute_histogram(frame, surrounding_mask, bins)


(10,)
(10,)


In [66]:
probabilities = np.zeros(frame.shape[:2])
probabilities.shape

(360, 640)

In [67]:

def calculateProbabilities(I_x, H_O, H_S):
    H_O_b=0
    H_S_b=0

    for a in I_x:

 
        H_O_b += H_O[a]
        H_S_b += H_S[a]


    total_hist = H_O_b + H_S_b

    return H_O_b / total_hist


In [68]:
def get_bin_indices(pixel_values, bins, value_range=256):
    return np.array([int((pixel_value * bins) // value_range) for pixel_value in pixel_values])


In [69]:
for y in range(frame.shape[0]):
    for x in range(frame.shape[1]):
            pixel_values = frame[y, x]
            bin_index = get_bin_indices(pixel_values, bins)
            if surrounding_mask[y, x] or object_mask[y, x]:
                probabilities[y, x] = calculateProbabilities(bin_index, H_O, H_S)
            else:
                probabilities[y, x] = 0.5



In [70]:
cv2.rectangle(probabilities, (left, top), (right, bottom), (0, 255, 0), 2)
cv2.imshow('Object Tracking', probabilities)
key = cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

In [71]:
initial_bbox_d = (352, 232, 23, 23) 

initial_hypothesis_d = np.array([initial_bbox[1], initial_bbox[0], 
                                   initial_bbox[1] + initial_bbox[3], 
                                   initial_bbox[0] + initial_bbox[2]])
top_d, left_d, bottom_d, right_d = initial_hypothesis

In [72]:
object_mask = np.zeros(frame.shape[:2], dtype=bool)
object_mask[top:bottom, left:right] = True


In [73]:
d_mask = np.zeros(frame.shape[:2], dtype=bool)
d_mask[top_d:bottom_d, left_d:right_d] = True

In [74]:
H_O = compute_histogram(frame, object_mask, bins) 
H_D = compute_histogram(frame, d_mask, bins)

(10,)
(10,)


In [75]:
probabilities_O_D = np.zeros(frame.shape[:2])
probabilities_O_D.shape

(360, 640)

In [76]:
for y in range(frame.shape[0]):
    for x in range(frame.shape[1]):
            pixel_values = frame[y, x]
            bin_index = get_bin_indices(pixel_values, bins)
            if d_mask[y, x] or object_mask[y, x]:
                probabilities_O_D[y, x] = calculateProbabilities(bin_index, H_O, H_D)
            else:
                probabilities_O_D[y, x] = 0.5

In [77]:
# cv2.rectangle(probabilities_O_D, (left_d, top_d), (right_d, bottom_d), (0, 255, 0), 2)
cv2.imshow('d Tracking', probabilities_O_D)
key = cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

In [ ]:
def combined_object_model(P_O_D, P_O_S, lambda_p):
    return lambda_p * P_O_D + (1 - lambda_p) * P_O_S

def update_object_model(P_current, P_previous, eta):
    return eta * P_current + (1 - eta) * P_previous


lambda_p = 0.5
eta = 0.1

combined_probabilities = combined_object_model(probabilities_O_D, probabilities, lambda_p)

updated_probabilities = update_object_model(combined_probabilities, probabilities, eta)

In [78]:
# def compute_vote_score(probabilities, candidate_bbox):
#     top, left, bottom, right = candidate_bbox
#     return np.sum(probabilities[top:bottom, left:right])

# def compute_distance_score(candidate_bbox, previous_center, sigma=1.0):
#     top, left, bottom, right = candidate_bbox
#     candidate_center = np.array([(top + bottom) / 2, (left + right) / 2])
#     distance = np.linalg.norm(candidate_center - previous_center)
#     return np.exp(-distance**2 / (2 * sigma**2))

# def localize_object(probabilities, previous_bbox, search_region, sigma=1.0):
#     top, left, bottom, right = search_region
#     previous_center = np.array([(previous_bbox[0] + previous_bbox[2]) / 2, 
#                                 (previous_bbox[1] + previous_bbox[3]) / 2])
    
#     best_score = -np.inf
#     best_bbox = None
    
#     for y in range(top, bottom):
#         for x in range(left, right):
#             candidate_bbox = (y, x, y + (previous_bbox[2] - previous_bbox[0]), x + (previous_bbox[3] - previous_bbox[1]))
#             if candidate_bbox[2] > probabilities.shape[0] or candidate_bbox[3] > probabilities.shape[1]:
#                 continue
#             vote_score = compute_vote_score(probabilities, candidate_bbox)
#             distance_score = compute_distance_score(candidate_bbox, previous_center, sigma)
#             score = vote_score * distance_score
#             if score > best_score:
#                 best_score = score
#                 best_bbox = candidate_bbox
    
#     return best_bbox

# # Define the search region proportional to the previous object location
# search_padding = 20
# search_region = (max(0, top - search_padding), max(0, left - search_padding), 
#                  min(frame.shape[0], bottom + search_padding), min(frame.shape[1], right + search_padding))

# # Localize the object in the new frame
# new_bbox = localize_object(updated_probabilities, initial_hypothesis, search_region)
# print("New object location:", new_bbox)